In [1]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
src_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
if src_dir not in sys.path:
    sys.path.append(src_dir)
from modules.preprocess import *
from modules.config_loader import load_config

In [2]:
model_df=pd.read_csv("../../data/standard_preprocessed_config_process_df.csv")
test_set_df=pd.read_csv("../../data/test_dataset.csv")
tracker_info=pd.read_excel("../../data/250409_tracker.xlsx")

In [3]:
config = load_config()
EQUIPMENTS_CATEGORY = config["EQUIPMENTS_CATEGORY"]
PARTS_CATEGORY = config["PARTS_CATEGORY"]
OTHERS_CATEGORY = config["OTHERS_CATEGORY"]

In [4]:
tracker_info["Process_NM"] = tracker_info["Process_NM"].fillna(tracker_info["Process(model Info)"])
tracker_info["Configuration"] = tracker_info["Configuration"].fillna(tracker_info["config(model Info)"])
tracker_info["Process_NM"] = tracker_info["Process_NM"].apply(lambda x: str(x)[:20] if pd.notna(x) else x)
tracker_info["Configuration"] = tracker_info["Configuration"].apply(lambda x: str(x)[:20] if pd.notna(x) else x)


In [5]:
tracker_info.drop(["EQUIP_ID","EQUIP_TX_CNT","Category_Code","USER_FAB_ID",
                   "COUNTRY_CD","CODE","Process(model Info)","config(model Info)"], axis=1, inplace=True)
tracker_info = tracker_info.dropna(subset=["MODEL_NAME"])

In [6]:
equipment_model_df=tracker_info[tracker_info['Category_Name'].isin(EQUIPMENTS_CATEGORY)]
parts_model_df=tracker_info[tracker_info['Category_Name'].isin(PARTS_CATEGORY)]
others_model_df=tracker_info[tracker_info['Category_Name'].isin(OTHERS_CATEGORY)]

In [7]:
others_model_df=others_model_df[(others_model_df["STD_MODEL_YN"]=="Y") & (others_model_df["P&C"]=="N")]
equipment_model_df=equipment_model_df[(equipment_model_df["STD_MODEL_YN"]=="Y") & (equipment_model_df["P&C"]=="N")]
equipment_model_df=pd.concat([equipment_model_df, others_model_df], ignore_index=True)
equipment_model_df

,Category_Name,MAKER_ID,MAKER_NAME,MODEL_ID,MODEL_NAME,Process_NM,Configuration,SERIAL_NO,VINTAGE,Tracker_availability,Seller,Owner,P&C,STD_MODEL_YN,STD_MAKER_YN
0,Stepper,MA1561,Nikon,MO1679,NSR-2205i12D,i線?????? LCD28,23℃、レチクル：6、Shrinc：Ty,1208257,NaN,UNSPEC,ESP,ESP,N,Y,Y
1,PKG ETC,MA1691,Panasonic,MO234067,FCB3 NM-SB50A,FCB3,NaN,2AB50011,NaN,AVAIL,ESP,ESP,N,Y,Y
2,Tester,MA18,ACETEC,MO15416,MRS-1000,NaN,NaN,ACE-063,NaN,UNSPEC,NaN,NaN,N,Y,Y
3,PKG ETC,MA2219,Takatori,MO33308,MWS-610,Wire Saw,NaN,1047-39,1998,UNSPEC,DAEGUN TRADING,DAEGUN TRADING,N,Y,Y
4,Metrology,MA261097,KLA,MO1291,KLA2139,Bright field,?Some missing parts(,876,NaN,UNSPEC,"BOT Lease Co.,Ltd","BOT Lease Co.,Ltd",N,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212744,Microscope,MA1639,Olympus,MO238760,BH3-MJLT4,??,Optical Microscope B,NaN,NaN,UNSPEC,CSMC Technologies Corporation,CSMC Technologies Corporation,N,Y,Y
212745,Microscope,MA1639,Olympus,MO73792,SZ61,Low Power Microscope,NaN,7C03050,NaN,AVAIL,ams Sensors Singapore Pte. Ltd.,ams Sensors Singapore Pte. Ltd.,N,Y,Y
212746,Microscope,MA1639,Olympus,MO73792,SZ61,Low Power Microscope,Low Power Scope -Ste,7D01660,NaN,AVAIL,ams Sensors Singapore Pte. Ltd.,ams Sensors Singapore Pte. Ltd.,N,Y,Y
212747,Microscope,MA1639,Olympus,MO73792,SZ61,Low Power Microscope,Binocular Stereo Zoo,NaN,NaN,AVAIL,ams Sensors Singapore Pte. Ltd.,ams Sensors Singapore Pte. Ltd.,N,Y,Y


In [8]:
result = equipment_model_df.groupby("MODEL_ID").agg(
    lambda x: ', '.join(set(map(str, x.dropna())))
).reset_index()
result

,MODEL_ID,Category_Name,MAKER_ID,MAKER_NAME,MODEL_NAME,Process_NM,Configuration,SERIAL_NO,VINTAGE,Tracker_availability,Seller,Owner,P&C,STD_MODEL_YN,STD_MAKER_YN
0,M000,Fab Others,MA000,Unknown,Unknown,"Laboratory table on , Photo Mask 검사기, Oscillat...","QTY : 20, Lot of (20, Volume (WxLxH)2500*2, SE...","WAJ-7480, 1341, 62997, KYD811606024, NSDE-212,...","1979, 2012, 2004, 2013, 1991, 2008, 2018, 1977...","UNSPEC, NOT_AVAIL, AVAIL, RUNNING","ASE Korea, Inc, SKhynix, Inc., Skyworks Soluti...","ASE Korea, Inc, SKhynix, Inc., Marvell Shangha...",N,Y,Y
1,MO1,Die Bonder,MA644,ESEC,2008 HS3,"Die Bonder, DIE ATTACH, Die bonding, Die Bondi...",weight : 900kg,"207574, 207586, 207575, 207576, 207587",,"NOT_AVAIL, AVAIL, UNSPEC","Siliconware Precision Ind. Co. Ltd., Dynamic M...","Siliconware Precision Ind. Co. Ltd., Dynamic M...",N,Y,Y
2,MO100,General Tester,MA10283,Agilent,4156A,"PARAMETER ANALYZER, PARAMETERANALYZER, Precisi...",", 전략물자유무 (ECCN Code) :, TESTR-SO","JP10A00510, JP10A00547, JP10A00661, JP10A00264...","1996, 1998, 1994","NOT_AVAIL, AVAIL, UNSPEC","주식회사 이지링크시스템, Intertec Sales Corp., United Sem...","주식회사 이지링크시스템, Freescale Kilbridge, Scotland, T...",N,Y,Y
3,MO1000,Stepper,MA321,Canon,FPA-1550M4W,"1/5G LINE STEPPER, Steppers, Stepper / Scanner...",", Remark : 10%?, A/E, Remark : 5%?, Tool...","301259M4, 7177, 200009, 201249M4, 12207, 11203...","1991, 2000, 1992, 1993, 1995, 1999","NOT_AVAIL, AVAIL, UNSPEC","MITSUBISHI HC Capital Inc. , Texas Instruments...","Texas Instruments Incorporated, Hightec System...",N,Y,Y
4,MO10003,Stepper,MA2052,Seiko Instrument Inc.,SMI8300,"scanning, FIB",,,1990,UNSPEC,"SDI Fabsurplus LLC, Sumitomo Mitsui Finance an...","ROHM Tsukuba Co.,Ltd",N,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14743,MO9950,Wire Bonder,MA2035,Shinkawa,UTC5100,"Wire Bonder, Bonder",Wire Bonder,"A1-70418, A1-70416, A1-70415, A1-70417, A1-704...",,"NOT_AVAIL, UNSPEC","Kaga Toshiba Electronics Corporation, Summit E...","Kaga Toshiba Electronics Corporation, Summit E...",N,Y,Y
14744,MO9960,Metrology,MA2039,Shinsung ENG,SGL-30,QE,,"1110006, 11-10-005",2011,"AVAIL, UNSPEC","SKhynix Semiconductor (China) Ltd, GEF","SKhynix Semiconductor (China) Ltd, GEF",N,Y,Y
14745,MO997,Stepper,MA321,Canon,FPA-1550M2,"CANON STEPPER(FPA#9), PHOTO(Stepper), PHOTO G-...",Stepper,411290,"1993, 1994, 1989","NOT_AVAIL, AVAIL, UNSPEC","The Innes Group, ELS Semi Co., Ltd., Magnachip...","ELS Semi Co., Ltd., ROHM Tsukuba Co.,Ltd, Magn...",N,Y,Y
14746,MO998,Stepper,MA321,Canon,FPA-1550M3-W,"PHOTO G-LINE STEPPER, ｇ線ステッパ, Stepper, PHOTO, ...","A/E, RETICLE SIZE：φ125ｍｍ, STEPPER Main Body/Fi","906183, 101382, 910325M3, 101380M3W, 003349M3,...","1991, 1990, 1989","NOT_AVAIL, AVAIL, UNSPEC","MITSUBISHI HC Capital Inc. , LUNS, Hangzhou E-...","Hangzhou E-Giant Semiconductor Tech Co., LTD, ...",N,Y,Y


In [9]:
result=result[result["MODEL_NAME"]!="Unknown"]


In [10]:
result

,MODEL_ID,Category_Name,MAKER_ID,MAKER_NAME,MODEL_NAME,Process_NM,Configuration,SERIAL_NO,VINTAGE,Tracker_availability,Seller,Owner,P&C,STD_MODEL_YN,STD_MAKER_YN
1,MO1,Die Bonder,MA644,ESEC,2008 HS3,"Die Bonder, DIE ATTACH, Die bonding, Die Bondi...",weight : 900kg,"207574, 207586, 207575, 207576, 207587",,"NOT_AVAIL, AVAIL, UNSPEC","Siliconware Precision Ind. Co. Ltd., Dynamic M...","Siliconware Precision Ind. Co. Ltd., Dynamic M...",N,Y,Y
2,MO100,General Tester,MA10283,Agilent,4156A,"PARAMETER ANALYZER, PARAMETERANALYZER, Precisi...",", 전략물자유무 (ECCN Code) :, TESTR-SO","JP10A00510, JP10A00547, JP10A00661, JP10A00264...","1996, 1998, 1994","NOT_AVAIL, AVAIL, UNSPEC","주식회사 이지링크시스템, Intertec Sales Corp., United Sem...","주식회사 이지링크시스템, Freescale Kilbridge, Scotland, T...",N,Y,Y
3,MO1000,Stepper,MA321,Canon,FPA-1550M4W,"1/5G LINE STEPPER, Steppers, Stepper / Scanner...",", Remark : 10%?, A/E, Remark : 5%?, Tool...","301259M4, 7177, 200009, 201249M4, 12207, 11203...","1991, 2000, 1992, 1993, 1995, 1999","NOT_AVAIL, AVAIL, UNSPEC","MITSUBISHI HC Capital Inc. , Texas Instruments...","Texas Instruments Incorporated, Hightec System...",N,Y,Y
4,MO10003,Stepper,MA2052,Seiko Instrument Inc.,SMI8300,"scanning, FIB",,,1990,UNSPEC,"SDI Fabsurplus LLC, Sumitomo Mitsui Finance an...","ROHM Tsukuba Co.,Ltd",N,Y,Y
5,MO10004,Metrology,MA2052,Seiko Instrument Inc.,SPA500,"AFM, INSPECTION","SCANNING ATOMIC FORC, ASSET No : 44000193-,",#10200187000065,"2004, 2003","AVAIL, UNSPEC","MITSUBISHI HC Capital Inc. , ROHM Co., Ltd, Tr...","ATREG, Samsung Electronics Co., Ltd., Sumitomo...",N,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14743,MO9950,Wire Bonder,MA2035,Shinkawa,UTC5100,"Wire Bonder, Bonder",Wire Bonder,"A1-70418, A1-70416, A1-70415, A1-70417, A1-704...",,"NOT_AVAIL, UNSPEC","Kaga Toshiba Electronics Corporation, Summit E...","Kaga Toshiba Electronics Corporation, Summit E...",N,Y,Y
14744,MO9960,Metrology,MA2039,Shinsung ENG,SGL-30,QE,,"1110006, 11-10-005",2011,"AVAIL, UNSPEC","SKhynix Semiconductor (China) Ltd, GEF","SKhynix Semiconductor (China) Ltd, GEF",N,Y,Y
14745,MO997,Stepper,MA321,Canon,FPA-1550M2,"CANON STEPPER(FPA#9), PHOTO(Stepper), PHOTO G-...",Stepper,411290,"1993, 1994, 1989","NOT_AVAIL, AVAIL, UNSPEC","The Innes Group, ELS Semi Co., Ltd., Magnachip...","ELS Semi Co., Ltd., ROHM Tsukuba Co.,Ltd, Magn...",N,Y,Y
14746,MO998,Stepper,MA321,Canon,FPA-1550M3-W,"PHOTO G-LINE STEPPER, ｇ線ステッパ, Stepper, PHOTO, ...","A/E, RETICLE SIZE：φ125ｍｍ, STEPPER Main Body/Fi","906183, 101382, 910325M3, 101380M3W, 003349M3,...","1991, 1990, 1989","NOT_AVAIL, AVAIL, UNSPEC","MITSUBISHI HC Capital Inc. , LUNS, Hangzhou E-...","Hangzhou E-Giant Semiconductor Tech Co., LTD, ...",N,Y,Y


In [67]:
result.to_csv("../../data/final_model_df.csv", index=False)
result=pd.read_csv("../../data/final_model_df.csv")